In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1881399,2021-11-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1881400,2021-11-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1881401,2021-11-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1881402,2021-11-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
48827,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48829,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48831,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48833,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48835,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1143955,2021-11-26,Arkansas,Arkansas,5001,3531,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1143957,2021-11-27,Arkansas,Arkansas,5001,3539,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1143959,2021-11-28,Arkansas,Arkansas,5001,3540,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1143961,2021-11-29,Arkansas,Arkansas,5001,3536,60.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1881399,2021-11-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1881400,2021-11-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1881401,2021-11-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1881402,2021-11-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-11-30') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
679,2021-11-30,Snohomish,Washington,72145,797.00,53061,WA,County,822083,8775.88,96.95
1356,2021-11-30,Cook,Illinois,677347,12051.00,17031,IL,County,5150233,13151.77,233.99
2032,2021-11-30,Orange,California,334398,5725.00,6059,CA,County,3175692,10529.93,180.28
2707,2021-11-30,Maricopa,Arizona,800696,12782.00,4013,AZ,County,4485414,17851.11,284.97
3382,2021-11-30,Los Angeles,California,1527836,27166.00,6037,CA,County,10039107,15218.84,270.60
...,...,...,...,...,...,...,...,...,...,...,...
1879871,2021-11-30,Wheeler,Oregon,114,1.00,41069,OR,County,1332,8558.56,75.08
1880285,2021-11-30,King,Texas,22,0.00,48269,TX,County,272,8088.24,0.00
1880668,2021-11-30,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10
1881047,2021-11-30,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
679,2021-11-30,Snohomish,Washington,72145,797.00,53061,WA,County,822083,8775.88,96.95,96.95,8775.88
1356,2021-11-30,Cook,Illinois,677347,12051.00,17031,IL,County,5150233,13151.77,233.99,233.99,13151.77
2032,2021-11-30,Orange,California,334398,5725.00,6059,CA,County,3175692,10529.93,180.28,180.28,10529.93
2707,2021-11-30,Maricopa,Arizona,800696,12782.00,4013,AZ,County,4485414,17851.11,284.97,284.97,17851.11
3382,2021-11-30,Los Angeles,California,1527836,27166.00,6037,CA,County,10039107,15218.84,270.60,270.60,15218.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879871,2021-11-30,Wheeler,Oregon,114,1.00,41069,OR,County,1332,8558.56,75.08,75.08,8558.56
1880285,2021-11-30,King,Texas,22,0.00,48269,TX,County,272,8088.24,0.00,0.00,8088.24
1880668,2021-11-30,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10,229.10,7101.95
1881047,2021-11-30,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1196592,2021-11-30,Hale,Alabama,3184,89.00,1065,AL,County,14651,21732.30,607.47,607.47,21732.30,1
1303437,2021-11-30,Clarke,Alabama,4848,86.00,1025,AL,County,23622,20523.24,364.07,364.07,20523.24,2
1195981,2021-11-30,Fayette,Alabama,3300,85.00,1057,AL,County,16302,20242.91,521.41,521.41,20242.91,3
1009606,2021-11-30,Winston,Alabama,4780,110.00,1133,AL,County,23629,20229.38,465.53,465.53,20229.38,4
694056,2021-11-30,Franklin,Alabama,6338,108.00,1059,AL,County,31362,20209.17,344.37,344.37,20209.17,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141514,2021-11-30,Converse,Wyoming,2322,42.00,56009,WY,County,13822,16799.31,303.86,303.86,16799.31,19
1006536,2021-11-30,Johnson,Wyoming,1357,16.00,56019,WY,County,8445,16068.68,189.46,189.46,16068.68,20
1142127,2021-11-30,Sublette,Wyoming,1486,23.00,56035,WY,County,9831,15115.45,233.95,233.95,15115.45,21
1421999,2021-11-30,Lincoln,Wyoming,2970,24.00,56023,WY,County,19830,14977.31,121.03,121.03,14977.31,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
934581,2021-11-30,Lowndes,Alabama,1782,68.00,1085,AL,County,9726,18322.02,699.16,699.16,18322.02,24,1
1196592,2021-11-30,Hale,Alabama,3184,89.00,1065,AL,County,14651,21732.30,607.47,607.47,21732.30,1,2
395556,2021-11-30,Walker,Alabama,12067,380.00,1127,AL,County,63521,18996.87,598.23,598.23,18996.87,13,3
827403,2021-11-30,Dallas,Alabama,5295,209.00,1047,AL,County,37196,14235.40,561.89,561.89,14235.40,62,4
933351,2021-11-30,Crenshaw,Alabama,2607,77.00,1041,AL,County,13772,18929.71,559.11,559.11,18929.71,16,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128401,2021-11-30,Sheridan,Wyoming,5838,57.00,56033,WY,County,30485,19150.40,186.98,186.98,19150.40,12,19
1386186,2021-11-30,Uinta,Wyoming,4047,31.00,56041,WY,County,20226,20008.90,153.27,153.27,20008.90,9,20
1421999,2021-11-30,Lincoln,Wyoming,2970,24.00,56023,WY,County,19830,14977.31,121.03,121.03,14977.31,22,21
930891,2021-11-30,Albany,Wyoming,6821,39.00,56001,WY,County,38880,17543.72,100.31,100.31,17543.72,17,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877045,2021-11-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1877046,2021-11-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1877047,2021-11-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1877048,2021-11-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1191628,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21732.30,2,1,1.00,nan
1191629,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21732.30,2,1,0.00,nan
1191630,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21732.30,2,1,0.00,nan
1191631,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21732.30,2,1,0.00,nan
1191632,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,607.47,21732.30,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446123,2021-11-26,Crook,Wyoming,984,20.00,56011,WY,County,7584,12974.68,263.71,276.90,13119.73,10,23,0.00,2.00
1446124,2021-11-27,Crook,Wyoming,984,20.00,56011,WY,County,7584,12974.68,263.71,276.90,13119.73,10,23,0.00,2.00
1446125,2021-11-28,Crook,Wyoming,984,20.00,56011,WY,County,7584,12974.68,263.71,276.90,13119.73,10,23,0.00,2.00
1446126,2021-11-29,Crook,Wyoming,993,20.00,56011,WY,County,7584,13093.35,263.71,276.90,13119.73,10,23,9.00,2.36


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
930839,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,24,1.00,1.43,0.00,nan
930840,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,24,0.00,1.50,0.00,nan
930841,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,24,0.00,1.36,0.00,nan
930842,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,24,0.00,1.50,0.00,nan
930843,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,24,0.00,1.43,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391174,2021-11-26,Teton,Wyoming,5366,14.00,56039,WY,County,23464,22869.08,59.67,59.67,22971.36,23,3,0.00,4.64,0.00,0.00
391175,2021-11-27,Teton,Wyoming,5366,14.00,56039,WY,County,23464,22869.08,59.67,59.67,22971.36,23,3,0.00,4.64,0.00,0.00
391176,2021-11-28,Teton,Wyoming,5366,14.00,56039,WY,County,23464,22869.08,59.67,59.67,22971.36,23,3,0.00,4.64,0.00,0.00
391177,2021-11-29,Teton,Wyoming,5377,14.00,56039,WY,County,23464,22915.96,59.67,59.67,22971.36,23,3,11.00,4.86,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-11-30') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
472652,2021-11-30,Imperial,California,38477,772.00,6025,CA,County,181215,21232.79,426.01,426.01,21232.79,1,3,31.00,67.07,0.00,0.50
234506,2021-11-30,San Bernardino,California,374543,5962.00,6071,CA,County,2180085,17180.20,273.48,273.48,17180.20,2,6,287.00,401.14,4.00,4.57
3382,2021-11-30,Los Angeles,California,1527836,27166.00,6037,CA,County,10039107,15218.84,270.60,270.60,15218.84,3,14,860.00,1008.29,28.00,18.79
1142669,2021-11-30,Tuolumne,California,7866,141.00,6109,CA,County,54478,14438.86,258.82,258.82,14438.86,4,19,21.00,12.64,0.00,0.29
104457,2021-11-30,Stanislaus,California,91548,1417.00,6099,CA,County,550660,16625.14,257.33,257.33,16625.14,5,8,83.00,100.86,2.00,1.36
830435,2021-11-30,Inyo,California,2566,46.00,6027,CA,County,18039,14224.74,255.00,255.00,14224.74,6,21,-2.00,6.21,0.00,0.29
72262,2021-11-30,Shasta,California,26081,451.00,6089,CA,County,180080,14483.01,250.44,250.44,14483.01,7,18,-1.00,40.86,7.00,2.71
752093,2021-11-30,Merced,California,45466,671.00,6047,CA,County,277680,16373.52,241.65,241.65,16373.52,8,9,49.00,51.43,1.00,1.21
83005,2021-11-30,San Joaquin,California,107559,1837.00,6077,CA,County,762148,14112.61,241.03,241.03,14112.61,9,22,32.00,100.43,0.00,2.43
1008320,2021-11-30,Kings,California,35206,365.00,6031,CA,County,152940,23019.48,238.66,238.66,23019.48,10,2,2.00,35.79,1.00,1.21


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1772881,2021-11-30,Lassen,California,7962,58.00,6035,CA,County,30573,26042.59,189.71,189.71,26042.59,19,1,39.00,7.14,1.00,0.29
1008320,2021-11-30,Kings,California,35206,365.00,6031,CA,County,152940,23019.48,238.66,238.66,23019.48,10,2,2.00,35.79,1.00,1.21
472652,2021-11-30,Imperial,California,38477,772.00,6025,CA,County,181215,21232.79,426.01,426.01,21232.79,1,3,31.00,67.07,0.00,0.50
129654,2021-11-30,Tulare,California,86087,1096.00,6107,CA,County,466195,18465.88,235.09,235.09,18465.88,11,4,17.00,89.93,1.00,4.07
295173,2021-11-30,Kern,California,157662,1804.00,6029,CA,County,900202,17514.07,200.40,200.40,17514.07,14,5,137.00,219.93,13.00,5.00
234506,2021-11-30,San Bernardino,California,374543,5962.00,6071,CA,County,2180085,17180.20,273.48,273.48,17180.20,2,6,287.00,401.14,4.00,4.57
47558,2021-11-30,Madera,California,26194,311.00,6039,CA,County,157327,16649.40,197.68,197.68,16649.40,16,7,14.00,36.07,0.00,1.29
104457,2021-11-30,Stanislaus,California,91548,1417.00,6099,CA,County,550660,16625.14,257.33,257.33,16625.14,5,8,83.00,100.86,2.00,1.36
752093,2021-11-30,Merced,California,45466,671.00,6047,CA,County,277680,16373.52,241.65,241.65,16373.52,8,9,49.00,51.43,1.00,1.21
1077698,2021-11-30,Glenn,California,4574,51.00,6021,CA,County,28393,16109.60,179.62,179.62,16109.60,21,10,8.00,2.71,0.00,0.36


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
472652,2021-11-30,Imperial,California,38477,772.00,6025,CA,County,181215,21232.79,426.01,426.01,21232.79,1,3,31.00,67.07,0.00,0.50
234506,2021-11-30,San Bernardino,California,374543,5962.00,6071,CA,County,2180085,17180.20,273.48,273.48,17180.20,2,6,287.00,401.14,4.00,4.57
3382,2021-11-30,Los Angeles,California,1527836,27166.00,6037,CA,County,10039107,15218.84,270.60,270.60,15218.84,3,14,860.00,1008.29,28.00,18.79
1142669,2021-11-30,Tuolumne,California,7866,141.00,6109,CA,County,54478,14438.86,258.82,258.82,14438.86,4,19,21.00,12.64,0.00,0.29
104457,2021-11-30,Stanislaus,California,91548,1417.00,6099,CA,County,550660,16625.14,257.33,257.33,16625.14,5,8,83.00,100.86,2.00,1.36
830435,2021-11-30,Inyo,California,2566,46.00,6027,CA,County,18039,14224.74,255.00,255.00,14224.74,6,21,-2.00,6.21,0.00,0.29
72262,2021-11-30,Shasta,California,26081,451.00,6089,CA,County,180080,14483.01,250.44,250.44,14483.01,7,18,-1.00,40.86,7.00,2.71
752093,2021-11-30,Merced,California,45466,671.00,6047,CA,County,277680,16373.52,241.65,241.65,16373.52,8,9,49.00,51.43,1.00,1.21
83005,2021-11-30,San Joaquin,California,107559,1837.00,6077,CA,County,762148,14112.61,241.03,241.03,14112.61,9,22,32.00,100.43,0.00,2.43
1008320,2021-11-30,Kings,California,35206,365.00,6031,CA,County,152940,23019.48,238.66,238.66,23019.48,10,2,2.00,35.79,1.00,1.21


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,33.93
1,03/21/20,Imperial,4,2.21,3,0.00,28.14
2,03/22/20,Imperial,4,2.21,3,0.00,24.50
3,03/23/20,Imperial,4,2.21,3,0.00,24.50
4,03/24/20,Imperial,9,4.97,3,5.00,24.86
...,...,...,...,...,...,...,...
6160,11/26/21,Glenn,4551,16028.60,10,0.00,2.71
6161,11/27/21,Glenn,4551,16028.60,10,0.00,1.93
6162,11/28/21,Glenn,4551,16028.60,10,0.00,1.93
6163,11/29/21,Glenn,4566,16081.43,10,15.00,2.50


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6269,11/26/21,Kings,363.00,237.35,10,0.00,1.29
6270,11/27/21,Kings,363.00,237.35,10,0.00,1.14
6271,11/28/21,Kings,363.00,237.35,10,0.00,1.14
6272,11/29/21,Kings,364.00,238.00,10,1.00,1.07


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)